In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### Store CSV into DataFrame

In [2]:
# https://www.kaggle.com/jayrav13/unemployment-by-county-us#output.csv
csv_file = "ETL Project/Resources/Unempoyment_rate.csv"
Unemployment_rate_df = pd.read_csv(csv_file)
Unemployment_rate_df=Unemployment_rate_df.loc[Unemployment_rate_df['Year']==2015]
Unemployment_rate_df=Unemployment_rate_df[Unemployment_rate_df.State != 'New York']
# Unemployment_rate_df=Unemployment_rate_df.drop(['Month'], axis=1)
Unemployment_rate_df=Unemployment_rate_df.rename(columns={"Rate": "Monthly_rate"})
Unemployment_rate_df.head()

,Year,Month,State,County,Monthly_rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [3]:
# Unemployment_rate_df.info()

In [4]:
# https://www.kaggle.com/muonneutrino/us-census-demographic-data
### Data for 2015 for Population
csv_file = "ETL Project/Resources/acs2015_county_data.csv"
county_data_df = pd.read_csv(csv_file)
# county_data_df.head()

In [5]:
###Create new data with select columns
### Data for 2015
county_col=["State","County","TotalPop","Men","Women"]
county_transformed = county_data_df[county_col].copy()

# Set index
# county_transformed.set_index("State", inplace=True)
county_transformed.reset_index()
county_transformed.head()

,State,County,TotalPop,Men,Women
0,Alabama,Autauga,55221,26745,28476
1,Alabama,Baldwin,195121,95314,99807
2,Alabama,Barbour,26932,14497,12435
3,Alabama,Bibb,22604,12073,10531
4,Alabama,Blount,57710,28512,29198


In [6]:
# county_transformed.info()

In [7]:
Unemployment_rate_merged=pd.merge(Unemployment_rate_df,county_transformed, on=('County'))
Unemployment_rate_merged=Unemployment_rate_merged.drop(['State_y'], axis=1)
Unemployment_rate_merged.head()

,Year,Month,State_x,County,Monthly_rate,TotalPop,Men,Women
0,2015,February,Nevada,Carson City,8.2,54482,27959,26523
1,2015,October,Nevada,Carson City,6.6,54482,27959,26523
2,2015,March,Nevada,Carson City,7.6,54482,27959,26523
3,2015,August,Nevada,Carson City,7.0,54482,27959,26523
4,2015,May,Nevada,Carson City,7.0,54482,27959,26523


## Store JSON data into a DataFrame

In [8]:
json_file = "ETL Project/Resources/Unempoyment_yearlry_rate.json"
# print(json_file)

In [9]:
filepath = json_file
with open(filepath) as jsonfile:
    json_norm = json.load(jsonfile)
# print(json_norm)

In [10]:
# json_normalize(json_norm)
# print(json.dumps(json_norm, indent=4))


In [11]:
json_2015 = json_norm['2015']['Annual']
print(json.dumps(json_2015, indent=4))



{
    "Alabama": {
        "12 Month Net Change": {},
        "Unemployment Rate": {
            "Autauga County": 5.2,
            "Baldwin County": 5.5,
            "Barbour County": 8.9,
            "Bibb County": 6.6,
            "Blount County": 5.4,
            "Bullock County": 7.8,
            "Butler County": 7.5,
            "Calhoun County": 7.0,
            "Chambers County": 6.0,
            "Cherokee County": 5.4,
            "Chilton County": 5.7,
            "Choctaw County": 9.1,
            "Clarke County": 11.8,
            "Clay County": 6.2,
            "Cleburne County": 6.0,
            "Coffee County": 5.9,
            "Colbert County": 7.9,
            "Conecuh County": 9.2,
            "Coosa County": 6.7,
            "Covington County": 7.1,
            "Crenshaw County": 6.0,
            "Cullman County": 5.3,
            "Dale County": 6.4,
            "Dallas County": 9.6,
            "Dekalb County": 6.0,
            "Elmore County": 5.1,
            "Esc

In [12]:
data={}
state_list=[]
for state in json_2015:
    data[state]=[]
    
    for county in json_2015[str(state)]['Unemployment Rate'].items():
        data[state].append(county)
        
for state in json_2015:
    state_list.append(state)
    
print( data[state])

[('Albany County', 3.1), ('Big Horn County', 4.3), ('Campbell County', 3.8), ('Carbon County', 3.6), ('Converse County', 3.6), ('Crook County', 3.4), ('Fremont County', 5.4), ('Goshen County', 3.2), ('Hot Springs County', 4.1), ('Johnson County', 4.8), ('Laramie County', 3.9), ('Lincoln County', 4.6), ('Natrona County', 4.8), ('Niobrara County', 2.9), ('Park County', 4.2), ('Platte County', 3.8), ('Sheridan County', 4.1), ('Sublette County', 5.1), ('Sweetwater County', 4.6), ('Teton County', 3.8), ('Uinta County', 4.9), ('Washakie County', 4.0), ('Weston County', 3.3)]


In [18]:
annual_rate_df=pd.DataFrame(data[state], columns=('County','Annual_rate'))
annual_rate_df.head()

,County,Annual_rate
0,Albany County,3.1
1,Big Horn County,4.3
2,Campbell County,3.8
3,Carbon County,3.6
4,Converse County,3.6


In [22]:
Unemployment_final_df=pd.merge(Unemployment_rate_merged,annual_rate_df, on=('County'), how='outer')


# Unemployment_rate_merged=pd.merge(Unemployment_rate_df,county_transformed, on=('County'))
# Unemployment_rate_merged=Unemployment_rate_merged.drop(['State_y'], axis=1)
Unemployment_final_df.head()

,Year,Month,State_x,County,Monthly_rate,TotalPop,Men,Women,Annual_rate
0,2015.0,February,Nevada,Carson City,8.2,54482.0,27959.0,26523.0,NaN
1,2015.0,October,Nevada,Carson City,6.6,54482.0,27959.0,26523.0,NaN
2,2015.0,March,Nevada,Carson City,7.6,54482.0,27959.0,26523.0,NaN
3,2015.0,August,Nevada,Carson City,7.0,54482.0,27959.0,26523.0,NaN
4,2015.0,May,Nevada,Carson City,7.0,54482.0,27959.0,26523.0,NaN


### Create New DataFrame and Clean DataFrame

In [24]:
Unemployment_final_df=pd.merge(Unemployment_rate_merged,annual_rate_df, on=('County'), how='outer')


# Unemployment_rate_merged=pd.merge(Unemployment_rate_df,county_transformed, on=('County'))
# Unemployment_rate_merged=Unemployment_rate_merged.drop(['State_y'], axis=1)
Unemployment_final_df.head()

,Year,Month,State_x,County,Monthly_rate,TotalPop,Men,Women,Annual_rate
0,2015.0,February,Nevada,Carson City,8.2,54482.0,27959.0,26523.0,NaN
1,2015.0,October,Nevada,Carson City,6.6,54482.0,27959.0,26523.0,NaN
2,2015.0,March,Nevada,Carson City,7.6,54482.0,27959.0,26523.0,NaN
3,2015.0,August,Nevada,Carson City,7.0,54482.0,27959.0,26523.0,NaN
4,2015.0,May,Nevada,Carson City,7.0,54482.0,27959.0,26523.0,NaN


In [23]:
Unemployment_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 9 columns):
Year            12 non-null float64
Month           12 non-null object
State_x         12 non-null object
County          35 non-null object
Monthly_rate    12 non-null float64
TotalPop        12 non-null float64
Men             12 non-null float64
Women           12 non-null float64
Annual_rate     23 non-null float64
dtypes: float64(6), object(3)
memory usage: 2.7+ KB


### Connect to database

In [25]:
class Unemployment(Base):
    __tablename__ = 'unemployment_rate'
    year= Column(Integer)
    state= Column(String(225))
    county= Column(String(225), primary_key=True)
    annual_un_rate= Column(float(225))
    woman= Column(Integer)
    men= Column(Integer)
    total_pop= Column(Integer)
    

NameError: name 'Column' is not defined

In [26]:
# Create Database connection

database_path= "unemployment_rate_db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [27]:
#Creat Metadata
Base.metadata.create_all(engine)

In [ ]:
# Create session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Add the records to the database
session.add()
session.commit()

### Check for tables